# Exploring Data

Technique #2 : Create interesting visualizations

In [ ]:
import csv
import numpy as np
import pandas as pd
import seaborn as sns
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
import matplotlib.pyplot as plt
from sklearn.feature_selection import SelectFromModel
from sklearn.tree import DecisionTreeClassifier # Import Decision Tree Classifier
from sklearn.model_selection import train_test_split # Import train_test_split function
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import RandomForestClassifier
from sklearn import metrics
from sklearn.metrics import confusion_matrix 
from sklearn.metrics import accuracy_score 
from sklearn.metrics import classification_report 
from sklearn.tree import export_graphviz
from sklearn.externals.six import StringIO  
from IPython.display import Image  
import sys
!{sys.executable} -m pip install pydotplus
import sys
!{sys.executable} -m pip install graphviz
from sklearn.naive_bayes import GaussianNB

In [ ]:
mydata= "finalDataset.csv"
df = pd.read_csv(mydata) 
df.head()
df.columns

In [ ]:
text = open("finalDataset.csv", "r")
text = ''.join([i for i in text]) \
    .replace("-", "")
x = open("finalDataset.csv","w")
x.writelines(text)
x.close()

df.head()

In [ ]:
#split dataset in target and other
target = ['deaths']
other = ['date', 'state', 'cases', 'abbreviation']

X = df[other]
y = df[target] 

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state=1) # 50% training and 20% test
X_train, X_validation, y_train, y_validation = train_test_split(X_train, y_train, test_size = 0.2, random_state=2)# 50% training and 20% Validation

#CHECK We expect the training features number of columns to match the testing feature number of columns and the number of rows to match for the respective training and testing features and the labels
print('X_train Shape of XTrain:', X_train.shape)
print('y_train Shape of YTrain:', y_train.shape)
print('X_test Shape of XTest:', X_test.shape)
print('y_test Shape of YTest:', y_test.shape)
print('Shape of XValidation:', X_validation.shape)
print('Shape of YValidation:', y_validation.shape)

In [ ]:
#SKLearn only takes # so I have to change all of the Strings to #
#use pd.concat to join the new columns with your original dataframe
df = pd.concat([df,pd.get_dummies(df['state'], prefix='state')],axis=1)
df = pd.concat([df,pd.get_dummies(df['abbreviation'], prefix='abbreviation')],axis=1)
                     
# get rid of the original column
df.drop(['state'],axis=1, inplace=True)
df.drop(['abbreviation'],axis=1, inplace=True)

#show all column names
for col in df.columns: 
    print(col)

In [ ]:
# Decision Tree classifer object
clf = DecisionTreeClassifier()

# Train Decision Tree Classifer
clf = clf.fit(X_train,y_train)

#Predict the response for test dataset
y_pred = clf.predict(X_test)

#ValueError: could not convert string to float: '?' 
    #for this reason I will now convert
    
results = confusion_matrix(y_test, y_pred) 
print('Confusion Matrix :')
print(results,"\n") 
print("Accuracy:", metrics.accuracy_score(y_test, y_pred),"\n")
print('Report : ')
print(classification_report(y_test, y_pred))

print(clf.feature_importances_) #use inbuilt class feature_importances of tree based classifiers
#plot graph of feature importances for better visualization
feat_importances = pd.Series(clf.feature_importances_, index=X.columns)
feat_importances.nlargest(10).plot(kind='barh')
plt.show()


In [ ]:
for state, importance in zip(df[other], clf.feature_importances_):
    print(name, "=", importance)

importances = clf.feature_importances_
indices = np.argsort(importances)
plt.title('Feature Importances')
plt.barh(range(len(indices)), importances[indices], color='b', align='center')
plt.yticks(range(len(indices)), [other[i] for i in indices])
plt.xlabel('Relative Importance')
plt.show()

In [ ]:
# Random Forest Classifier
clf = RandomForestClassifier(n_estimators=100)

#Train the model using the training sets y_pred=clf.predict(X_test)
clf.fit(X_train,y_train)
y_pred = clf.predict(X_test)

results = confusion_matrix(y_test, y_pred) 
print('Confusion Matrix :')
print(results,"\n") 
print("Accuracy:", metrics.accuracy_score(y_test, y_pred),"\n")
print('Report : ')
print(classification_report(y_test, y_pred))

print(clf.feature_importances_) #use inbuilt class feature_importances of tree based classifiers
#plot graph of feature importances for better visualization
feat_importances = pd.Series(clf.feature_importances_, index=X.columns)
feat_importances.nlargest(10).plot(kind='barh')
plt.show()

In [ ]:
# BETTER Decision Tree classifer object
clf = DecisionTreeClassifier(criterion="entropy", max_depth=9)

# Train Decision Tree Classifer
clf = clf.fit(X_train,y_train)

#Predict the response for test dataset
y_pred = clf.predict(X_test)

results = confusion_matrix(y_test, y_pred) 
print('Confusion Matrix :')
print(results,"\n") 
print("Accuracy:", metrics.accuracy_score(y_test, y_pred),"\n")
print('Report : ')
print(classification_report(y_test, y_pred))

print(clf.feature_importances_) #use inbuilt class feature_importances of tree based classifiers
#plot graph of feature importances for better visualization
feat_importances = pd.Series(clf.feature_importances_, index=X.columns)
feat_importances.nlargest(10).plot(kind='barh')
plt.show()

In [ ]:
#apply SelectKBest class to extract top 10 best features
bestfeatures = SelectKBest(score_func=chi2, k=10)
fit = bestfeatures.fit(X_train,y_train)
dfscores = pd.DataFrame(fit.scores_)
dfcolumns = pd.DataFrame(X.columns)
#concat two dataframes for better visualization 
featureScores = pd.concat([dfcolumns,dfscores],axis=1)
featureScores.columns = ['Specs','Score']  #naming the dataframe columns
print(featureScores.nlargest(10,'Score'))  #print 10 best features